In [1]:
import cv2
import numpy as np
import torch
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from threading import Thread
from queue import Queue
from collections import deque

# Load the gender classification model (fine-tuned MobileNetV2)
model = load_model('Models/gender_classification_model_mobilenetv2.h5')

# Load the YOLOv5 model for person detection
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Video file or webcam
video_path = 0 # use 0 for webcam or provide path to video

# Function to preprocess image for gender classification
def preprocess_for_classification(image):
    image = cv2.resize(image, (224, 224))
    image = preprocess_input(image)
    image = np.expand_dims(image, axis=0)
    return image

# Function to calculate IoU (Intersection over Union) for two bounding boxes
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

# Threaded frame capture class
class FrameCapture(Thread):
    def __init__(self, src, queue):
        super().__init__()
        self.cap = cv2.VideoCapture(src)
        self.queue = queue
        self.stopped = False

    def run(self):
        while not self.stopped:
            ret, frame = self.cap.read()
            if not ret:
                self.stop()
                break
            if self.queue.qsize() < 10:
                self.queue.put(frame)

    def stop(self):
        self.stopped = True
        self.cap.release()

# Start frame capture thread
frame_queue = Queue()
capture_thread = FrameCapture(video_path, frame_queue)
capture_thread.start()

frame_count = 0
tracking_data = {}
next_person_id = 0

# Process video frames
while True:
    if not frame_queue.empty():
        frame = frame_queue.get()
        frame_count += 1

        if frame_count % 5 == 0:  # Perform detection every 5 frames
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = model_yolo(img_rgb)
            new_detections = []

            for *xyxy, conf, cls in results.xyxy[0].cpu().numpy():
                if cls == 0 and conf > 0.5:  # Class 0 is 'person'
                    x, y, w, h = int(xyxy[0]), int(xyxy[1]), int(xyxy[2]-xyxy[0]), int(xyxy[3]-xyxy[1])
                    new_detections.append([x, y, w, h, conf])

            updated_tracking_data = {}
            for detection in new_detections:
                x, y, w, h, confidence = detection
                matched = False

                for person_id, data in tracking_data.items():
                    tracked_box = data['box']
                    iou = calculate_iou(tracked_box, (x, y, w, h))

                    if iou > 0.5:
                        data['box'] = (x, y, w, h)
                        data['frames_seen'] = frame_count
                        updated_tracking_data[person_id] = data
                        matched = True
                        break

                if not matched:
                    updated_tracking_data[next_person_id] = {
                        'box': (x, y, w, h),
                        'gender_votes': deque(maxlen=15),  # Increased window for votes
                        'fixed_gender': None,
                        'frames_seen': frame_count
                    }
                    next_person_id += 1

            tracking_data = updated_tracking_data

        # Process each tracked person
        for person_id, data in list(tracking_data.items()):
            x, y, w, h = data['box']
            person_img = frame[y:y+h, x:x+w]

            if data['fixed_gender'] is None:
                input_img = preprocess_for_classification(person_img)
                gender_pred = model.predict(input_img)[0][0]
                gender = 'Male' if gender_pred < 0.5 else 'Female'

                # Debugging: Print the prediction confidence
                print(f"Person {person_id}: Gender prediction = {gender_pred:.2f}, Classified as = {gender}")

                data['gender_votes'].append(gender)

                # Use the most frequent gender prediction after collecting enough votes
                if len(data['gender_votes']) >= data['gender_votes'].maxlen:
                    vote_count = np.array(data['gender_votes'])
                    male_votes = np.sum(vote_count == 'Male')
                    female_votes = np.sum(vote_count == 'Female')

                    if male_votes > female_votes:
                        data['fixed_gender'] = 'Male'
                    elif female_votes > male_votes:
                        data['fixed_gender'] = 'Female'
                    else:
                        data['fixed_gender'] = 'Uncertain'

            else:
                gender = data['fixed_gender']

            color = (0, 255, 0) if gender == 'Male' else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f'{gender} ({frame_count - data["frames_seen"]} frames)', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            if frame_count - data['frames_seen'] > 150:
                del tracking_data[person_id]

        cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
capture_thread.stop()
cv2.destroyAllWindows()


2024-08-16 13:10:42.207677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 13:10:42.220019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 13:10:42.223567: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 13:10:42.234095: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-16 13:10:43.041092: W tensorflow/compiler/tf2

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Person 0: Gender prediction = 0.44, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Person 0: Gender prediction = 0.33, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Person 0: Gender prediction = 0.36, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Person 0: Gender prediction = 0.37, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


I0000 00:00:1723794051.358208    9583 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Person 0: Gender prediction = 0.38, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Person 0: Gender prediction = 0.49, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Person 0: Gender prediction = 0.41, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Person 0: Gender prediction = 0.41, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Person 0: Gender prediction = 0.40, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Person 0: Gender prediction = 0.45, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Person 0: Gender prediction = 0.36, Classified as = Male
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Person 0: Gender prediction = 0.51, Classified as = Female
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Person 0: Gender prediction = 0.51, Classified as = Female
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Person 0: Gender prediction = 0.53, Classified as = Female
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Person 0: Gender prediction = 0.50, Classifi

In [1]:
!pip install pipreqs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 12.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 30.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=168dbd3885192e77535fb3488f694dbb88251139a02389d8ba6d07b05bff1fee
  Stored in directory: /home/shanks/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: ipython
    Found existing installation: ipython 8.25.0
    Uninstalling ipython-8.25.0:
      Successfully uninstalled ipython-8.25.0
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 7.10.0
    Uninstalling nbconvert-7.10.0:
      Successfully uninstalled nbconvert-7.10.0


In [7]:
!pipreqs --scan-notebooks --force

Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in /home/shanks/data-science/CCTV-cam/requirements.txt
